In [ ]:
%pwd
%cd ../..

In [ ]:
# %env OMP_NUM_THREADS=1
# %env OMP_NUM_THREADS

In [ ]:
# file_name = 'session_0398bd16_boey_base/poke_10485760_steps'
file_name = r'D:\pokered\running\session_f74031ee_env19_lr3e-4_ent01_bs2048_ep3_5120_vf05_v2_es2_fullrun_gamma96_ppo29_1058m\poke_49152000_steps'
run_cnt = 24
ep_length_multiplier = 3_000  # how many k steps per episode
# device = 'cpu'
device = 'cuda'

In [ ]:
from os.path import exists
from pathlib import Path
import uuid
from baselines.boey_baselines2.red_gym_env import RedGymEnvV3 as RedGymEnv
from stable_baselines3 import A2C, PPO
from stable_baselines3.common import env_checker
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.callbacks import CheckpointCallback
import timeit

def make_env(rank, env_conf, seed=0):
    """
    Utility function for multiprocessed env.
    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the initial seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env_config['env_id'] = rank
        env = RedGymEnv(env_conf)
        env.reset(seed=(seed + rank))
        return env
    set_random_seed(seed)
    return _init

sess_path = Path(f'session_{str(uuid.uuid4())[:8]}session_3cf41181_env19_lr3e-4_ent01_bs2048_ep3_5120_vf05_v2_es2_fullrun_gamma96_ppo29_1.1b')
ep_length = 1024 * ep_length_multiplier

# state_dir = Path('../saved_states_v5')
state_dir = Path(r'D:\pokered\states\env19')
env_config = {
                'headless': True, 'save_final_state': True, 
                'early_stop': True,  # resumed early stopping to ensure reward signal
                'action_freq': 24, 'init_state': 'has_pokedex_nballs_noanim.state', 'max_steps': ep_length, 
                # 'env_max_steps': env_max_steps,
                'print_rewards': True, 'save_video': True, 'fast_video': True, 'session_path': sess_path,
                'gb_path': 'PokemonRed.gb', 'debug': False, 'sim_frame_dist': 2_000_000.0, 
                'use_screen_explore': False, 'reward_scale': 4, 
                'extra_buttons': False, 'restricted_start_menu': False, 
                'noop_button': True,
                'swap_button': True,
                'enable_item_manager': True,
                'level_reward_badge_scale': 1.0,
                # 'randomize_first_ep_split_cnt': num_cpu,
                # 'start_from_state_dir': state_dir, 
                # 'save_state_dir': state_dir,
                'explore_weight': 1.5, # 3
                'special_exploration_scale': 1.0,  # double the exploration for special maps (caverns)
                'enable_stage_manager': True,
                'enable_item_purchaser': True,
                'auto_skip_anim': True,
                'auto_skip_anim_frames': 8,
                'early_stopping_min_reward': 2.0,
                # 'total_envs': num_cpu,
                'level_manager_eval_mode': True,
        }

num_cpu = 1 #64 #46  # Also sets the number of episodes per training iteration
# envs = make_env(0, env_config)() #SubprocVecEnv([make_env(i, env_config) for i in range(num_cpu)])
envs = SubprocVecEnv([make_env(i, env_config) for i in range(run_cnt)])
# envs = DummyVecEnv([make_env(i, env_config) for i in range(run_cnt)])

#env_checker.check_env(env)
# file_name = 'session_bec9c90f/poke_45875200_steps'

print('\nloading checkpoint')
model = PPO.load(file_name, env=envs, custom_objects={'lr_schedule': 0, 'clip_range': 0}, device=device)
# model = PPO.load(file_name, env=envs, custom_objects={'lr_schedule': 0, 'clip_range': 0}, device=device)

#keyboard.on_press_key("M", toggle_agent)

In [ ]:
print(model.policy)

In [ ]:
model.n_steps

In [ ]:
if False:
    print('a')

In [ ]:
all_rewards = []
obses = envs.reset()
start = timeit.default_timer()

In [ ]:
for i in range(1):
    print(f'\nrun {i}')
    for cur_step in range(ep_length):
        # action = 7 # pass action
        action, _states = model.predict(obses, deterministic=False)
        # print(f'step {cur_step}, action: {action}')
        obses, rewards, terminated, info = envs.step(action)
        all_rewards.append(rewards[0])
        if cur_step and cur_step % 2000 == 0:
            print(f'step {cur_step}, fps: {cur_step / (timeit.default_timer() - start):.2f}')
            print(f'rewards sum: {sum(all_rewards)}, max: {max(all_rewards)}, min: {min(all_rewards)}')
            all_rewards = []

In [ ]:
# call env reset to release the video writer handle
envs.reset()

In [ ]:
total_params = 0
for name, parameter in model.policy.named_parameters():
    if not parameter.requires_grad:
        continue
    param = parameter.numel()
    print(name, param)
    total_params+=param
print(f'total params: {total_params}')